In [1]:
import datetime
import pandas as pd
import numpy as np
import yfinance as yf


# for plotting purposes
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# Equivalent timeframe mappings
frame_M1 = '1m'
frame_M15 = '15m'
frame_M30 = '30m'
frame_H1 = '1h'
frame_H4 = '4h'
frame_D1 = '1d'
frame_W1 = '1wk'


# Matching your MT5 asset list with Yahoo tickers
assets = {
    'EURUSD': 'EURUSD=X',
    'USDCHF': 'USDCHF=X',
    'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X',
    'BTCUSD': 'BTC-USD',
    'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X',
    'XAGUSD': 'XAGUSD=X',
    'SP500m': '^GSPC',
    'UK100': '^FTSE'
}

In [3]:
def get_quotes(time_frame,  period='2yrs', asset='EURUSD'):
    try:
        symbol = assets[asset]
        df = yf.download(
            symbol,
            period=period,
            interval=time_frame,
        )

        if df.empty:
            print(f"No data for {asset}. Check ticker symbol or timeframe.")
            return pd.DataFrame()

        df.reset_index(inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching data for {asset}: {e}")
        return pd.DataFrame()


def mass_import(asset_index, time_frame):

    asset_list = list(assets.keys())
    asset = asset_list[asset_index]

    frame_M15 = '15m'
    frame_M30 = '30m'
    frame_H1 = '1h'
    frame_H4 = '4h'
    frame_D1 = '1d'
    frame_W1 = '1wk'

    if time_frame == '1WK':
        data = get_quotes(frame_W1, '10y', asset=asset)
    elif time_frame == 'D1':
        data = get_quotes(frame_D1, '5y', asset=asset)
    elif time_frame == '4H':
        data = get_quotes(frame_H4, '2y', asset=asset)
    elif time_frame == '1H':
        data = get_quotes(frame_H1, '2y', asset=asset)
    elif time_frame == 'M30':
        data = get_quotes(frame_M30, '60d', asset=asset)
    elif time_frame == 'M15':
        data = get_quotes(frame_M15, '60d', asset=asset)
    else:
        data = get_quotes(frame_D1, 'D1', asset=asset)

    if data.empty:
        print(f"No data found for {asset} on timeframe {time_frame}")
        return np.array([])

    return data


def detect_key_levels(df, current_candle, backcandles=50, test_candles=10):
    """
    Detects key support and resistance levels in a given backcandles window.
    
    A level is identified if a candle's high is the highest or its low is the lowest 
    compared to `test_candles` before and after it.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'High' and 'Low' columns.
        current_candle (int): The index of the current candle (latest available candle).
        backcandles (int): Number of candles to look back.
        test_candles (int): Number of candles before and after each candle to check.

    Returns:
        dict: A dictionary with detected 'support' and 'resistance' levels.
    """
    key_levels = {"support": [], "resistance": []}

    # Define the last candle that can be tested to avoid lookahead bias
    last_testable_candle = current_candle - test_candles

    # Ensure we have enough data
    if last_testable_candle < backcandles + test_candles:
        return key_levels  # Not enough historical data

    # Iterate through the backcandles window
    for i in range(current_candle - backcandles, last_testable_candle):
        high = df['High'].iloc[i]
        low = df['Low'].iloc[i]

        # Get surrounding window of test_candles before and after
        before = df.iloc[max(0, i - test_candles):i]
        after = df.iloc[i + 1: min(len(df), i + test_candles + 1)]

        # Check if current high is the highest among before & after candles
        if high > before['High'].max() and high > after['High'].max():
            key_levels["resistance"].append((i, high))

        # Check if current low is the lowest among before & after candles
        if low < before['Low'].min() and low < after['Low'].min():
            key_levels["support"].append((i, low))

    return key_levels

def fill_key_levels(df, backcandles=50, test_candles=10):
    """
    Adds a 'key_levels' column to the DataFrame where each row contains all
    key support and resistance levels detected up to that candle (including
    both the level value and the index of the candle that generated it).
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'High' and 'Low' columns.
        backcandles (int): Lookback window for detecting key levels.
        test_candles (int): Number of candles before/after for validation.

    Returns:
        pd.DataFrame: Updated DataFrame with the new 'key_levels' column.
    """
    df["key_levels"] = None  # Initialize the column

    from tqdm import tqdm
    for current_candle in tqdm(range(backcandles + test_candles, len(df))):
        # Detect key levels for the current candle
        key_levels = detect_key_levels(
            df, current_candle, backcandles, test_candles)

        # Collect support and resistance levels (with their indices) up to current_candle
        support_levels = [(idx, level) for (idx, level) in key_levels["support"]
                          if idx < current_candle]
        resistance_levels = [(idx, level) for (idx, level) in key_levels["resistance"]
                             if idx < current_candle]

        # Store the levels along with the originating candle index
        if support_levels or resistance_levels:
            df.at[current_candle, "key_levels"] = {
                "support": support_levels,
                "resistance": resistance_levels
            }

    return df


def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes the 'Ticker' row level from a Yahoo Finance DataFrame
    that has multi-index columns like ('EURUSD=X', 'Open').

    Returns a DataFrame with simple column names like 'Open', 'High', etc.
    """
    # If columns are MultiIndex (e.g. ('EURUSD=X', 'Open')), flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df

In [4]:
dataF = mass_import(0, '1H')
dataF = dataF.reset_index()            # Move Datetime from index to column
dataF.rename(columns={'Date': 'Gmt time'}, inplace=True)


try:
    my_data = dataF[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']]
except:
    my_data = dataF[['Gmt time', 'Open', 'High', 'Low', 'Close', 'Volume']]


my_data = flatten_yf_columns(my_data)
my_data

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_38590/1428025713.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed


Price,Datetime,Open,High,Low,Close,Volume
0,2023-11-24 20:00:00+00:00,1.094931,1.095170,1.094691,1.094811,0
1,2023-11-24 21:00:00+00:00,1.094811,1.094931,1.094331,1.094571,0
2,2023-11-24 22:00:00+00:00,1.094331,1.094331,1.094331,1.094331,0
3,2023-11-27 00:00:00+00:00,1.094331,1.094331,1.093135,1.093255,0
4,2023-11-27 01:00:00+00:00,1.093135,1.095290,1.093016,1.094931,0
...,...,...,...,...,...,...
12327,2025-11-24 16:00:00+00:00,1.153403,1.153403,1.152206,1.152206,0
12328,2025-11-24 17:00:00+00:00,1.152206,1.152339,1.151543,1.152206,0
12329,2025-11-24 18:00:00+00:00,1.152074,1.153136,1.152074,1.153136,0
12330,2025-11-24 19:00:00+00:00,1.153004,1.153270,1.152605,1.153136,0


In [5]:
df = fill_key_levels(my_data, backcandles=50, test_candles=10)

100%|██████████| 12272/12272 [00:26<00:00, 471.94it/s]


In [6]:
df

Price,Datetime,Open,High,Low,Close,Volume,key_levels
0,2023-11-24 20:00:00+00:00,1.094931,1.095170,1.094691,1.094811,0,None
1,2023-11-24 21:00:00+00:00,1.094811,1.094931,1.094331,1.094571,0,None
2,2023-11-24 22:00:00+00:00,1.094331,1.094331,1.094331,1.094331,0,None
3,2023-11-27 00:00:00+00:00,1.094331,1.094331,1.093135,1.093255,0,None
4,2023-11-27 01:00:00+00:00,1.093135,1.095290,1.093016,1.094931,0,None
...,...,...,...,...,...,...,...
12327,2025-11-24 16:00:00+00:00,1.153403,1.153403,1.152206,1.152206,0,"{'support': [(12304, 1.1494252681732178)], 're..."
12328,2025-11-24 17:00:00+00:00,1.152206,1.152339,1.151543,1.152206,0,"{'support': [(12304, 1.1494252681732178)], 're..."
12329,2025-11-24 18:00:00+00:00,1.152074,1.153136,1.152074,1.153136,0,"{'support': [(12304, 1.1494252681732178)], 're..."
12330,2025-11-24 19:00:00+00:00,1.153004,1.153270,1.152605,1.153136,0,"{'support': [(12304, 1.1494252681732178)], 're..."


In [7]:
def plot_fvg_and_key_levels(df, start_idx, end_idx, extension=30):
    """
    Plots candlesticks, FVG zones, and key levels (support/resistance) for a
    subset of a DataFrame from `start_idx` to `end_idx`.
    
    The FVG column is expected to have tuples of the form:
        (fvg_type, start_price, end_price, trigger_index)

    The key_levels column is expected to have dictionaries of the form:
        {
          "support": [(idx, price), (idx, price), ...],
          "resistance": [(idx, price), (idx, price), ...]
        }

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain: "Open", "High", "Low", "Close", "FVG", "key_levels".
    start_idx : int
        Starting row index for plotting.
    end_idx : int
        Ending row index for plotting.
    extension : int
        How far (in x-axis units/index steps) to extend the FVG rectangles
        and key-level lines.
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        A Plotly Figure with the candlesticks, FVG, and key-level lines.
    """

    # Slice the DataFrame to the desired plotting range
    # dfpl = df.loc[start_idx:end_idx]
    dfpl = df.tail(200)

    # Create the figure
    fig = go.Figure()

    # -- 1) Add Candlestick Chart --
    fig.add_trace(go.Candlestick(
        x=dfpl.index,
        open=dfpl["Open"],
        high=dfpl["High"],
        low=dfpl["Low"],
        close=dfpl["Close"],
        name="Candles"
    ))

    # -- 2) Add FVG Zones --
    for i, row in dfpl.iterrows():
        # Check if "FVG" is a valid tuple: (fvg_type, start_price, end_price, trigger_index)
        if isinstance(row.get("FVG"), tuple):
            fvg_type, start_price, end_price, trigger_idx = row["FVG"]

            # Choose a fill color based on bullish vs. bearish
            if fvg_type == "bullish":
                color = "rgba(0, 255, 0, 0.3)"   # greenish
            else:
                color = "rgba(255, 0, 0, 0.3)"   # reddish

            fig.add_shape(
                type="rect",
                x0=trigger_idx,
                x1=trigger_idx + extension,
                y0=start_price,
                y1=end_price,
                fillcolor=color,
                opacity=0.4,
                layer="below",
                line=dict(width=0),
            )

    # -- 3) Add Key Levels as Horizontal Lines --
    for i, row in dfpl.iterrows():
        key_levels = row.get("key_levels", None)
        if key_levels:
            # key_levels is a dict: {"support": [(idx, val), ...], "resistance": [(idx, val), ...]}
            support_levels = key_levels.get("support", [])
            resistance_levels = key_levels.get("resistance", [])

            # Plot support levels
            for (gen_idx, s_price) in support_levels:
                # We only draw the line if gen_idx is in (start_idx, end_idx)
                # You can decide to relax/omit this check if you want lines from outside the window.
                if start_idx <= gen_idx <= end_idx:
                    fig.add_shape(
                        type="line",
                        x0=gen_idx,
                        x1=gen_idx + extension,
                        y0=s_price,
                        y1=s_price,
                        line=dict(color="blue", width=2),
                        layer="below"
                    )

            # Plot resistance levels
            for (gen_idx, r_price) in resistance_levels:
                if start_idx <= gen_idx <= end_idx:
                    fig.add_shape(
                        type="line",
                        x0=gen_idx,
                        x1=gen_idx + extension,
                        y0=r_price,
                        y1=r_price,
                        line=dict(color="orange", width=2),
                        layer="below"
                    )

    # -- 4) Figure Aesthetics --
    fig.update_layout(
        width=1200,
        height=800,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        plot_bgcolor='black',
        paper_bgcolor='black'
    )

    fig.show()
    return fig


fig = plot_fvg_and_key_levels(df, start_idx=12173, end_idx=12373, extension=30)
